In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

#读取数据
df = pd.read_csv("/input/data/train_data.csv")  
data = np.array(df['铁路客运量_当期值(万人)'])

In [3]:
# 数据预处理
normalized_data = (data - np.mean(data)) / np.std(data)

seq_size = 3
train_x, train_y = [], []
for i in range(len(normalized_data) - seq_size - 1):
    train_x.append(np.expand_dims(normalized_data[i : i + seq_size], axis=1).tolist())
    train_y.append(normalized_data[i + 1 : i + seq_size + 1].tolist())

input_dim = 1
X = tf.placeholder(tf.float32, [None, seq_size, input_dim])
Y = tf.placeholder(tf.float32, [None, seq_size])

In [4]:
# regression
def ass_rnn(hidden_layer_size=6):
    W = tf.Variable(tf.random_normal([hidden_layer_size, 1]), name='W')
    b = tf.Variable(tf.random_normal([1]), name='b')
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_layer_size)
    outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    W_repeated = tf.tile(tf.expand_dims(W, 0), [tf.shape(X)[0], 1, 1])
    out = tf.batch_matmul(outputs, W_repeated) + b
    out = tf.squeeze(out)
    return out

In [5]:
#训练模型
def train_rnn():
    out = ass_rnn()
    loss = tf.reduce_mean(tf.square(out - Y))
    train_op = tf.train.AdamOptimizer(learning_rate=0.003).minimize(loss)

    saver = tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        #tf.get_variable_scope().reuse_variables()
        sess.run(tf.global_variables_initializer())
        for step in range(10000):
            _, loss_ = sess.run([train_op, loss], feed_dict={X: train_x, Y: train_y})
            if step % 10 == 0:
                # 用测试数据评估loss
                print("第%s步：loss:%f"%(step, loss_))
        print("保存模型: ", saver.save(sess, 'output/ass.model'))

train_rnn()

第0步：loss:1.443983
第10步：loss:1.147347
第20步：loss:0.897415
第30步：loss:0.705665
第40步：loss:0.572029
第50步：loss:0.483894
第60步：loss:0.425597
第70步：loss:0.386313
第80步：loss:0.360134
第90步：loss:0.342122
第100步：loss:0.328143
第110步：loss:0.315995
第120步：loss:0.304823
第130步：loss:0.294254
第140步：loss:0.284098
第150步：loss:0.274266
第160步：loss:0.264716
第170步：loss:0.255444
第180步：loss:0.246463
第190步：loss:0.237807
第200步：loss:0.229514
第210步：loss:0.221627
第220步：loss:0.214189
第230步：loss:0.207239
第240步：loss:0.200808
第250步：loss:0.194922
第260步：loss:0.189592
第270步：loss:0.184822
第280步：loss:0.180600
第290步：loss:0.176905
第300步：loss:0.173705
第310步：loss:0.170961
第320步：loss:0.168626
第330步：loss:0.166654
第340步：loss:0.164997
第350步：loss:0.163609
第360步：loss:0.162447
第370步：loss:0.161472
第380步：loss:0.160653
第390步：loss:0.159960
第400步：loss:0.159372
第410步：loss:0.158868
第420步：loss:0.158434
第430步：loss:0.158057
第440步：loss:0.157729
第450步：loss:0.157440
第460步：loss:0.157186
第470步：loss:0.156960
第480步：loss:0.156759
第490步：loss:0.156579
第500步：loss: